In [1]:
import pandas as pd
import numpy as np
import pathlib
import loguru

from src.dataset import DatasetLoader
from src.dataset import DatasetEnum
from src.pipeline import ModelPipeline

In [2]:
data_loader = DatasetLoader(pathlib.Path("./datasets"))

data = data_loader.load_dataset(DatasetEnum.weather)

features = [col_name for col_name in data.columns if col_name not in ["target", "Date", "Location", "RainToday"]]

for col_name in features:
    if data[col_name].dtypes == "float" and np.all(data[~data[col_name].isna()][col_name] % 1.0 == 0.0):
        data[col_name] = data[col_name].fillna(np.round(np.mean(data[col_name]))).astype("int32")
    elif data[col_name].dtypes == "float":
        data[col_name] = data[col_name].fillna(np.mean(data[col_name])).astype("float64")
    elif data[col_name].dtypes == "category":
        data[col_name] = data[col_name].astype("str").fillna("unk").astype("category")

data.head(3)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,target
36974,2010-03-22,WaggaWagga,8.4,25.3,0.0,5.400000,10.600000,SW,31,W,...,76,30,1020.5,1018.6,1,1,14.4,24.3,No,0
141221,2014-07-09,Darwin,17.5,30.7,0.0,5.000000,10.400000,ESE,30,SSE,...,68,39,1013.7,1010.4,6,6,22.3,29.6,No,0
51663,2016-05-23,Tuggeranong,6.2,17.5,0.0,5.463885,7.652609,WSW,57,WNW,...,68,35,1014.3,1015.4,4,4,15.7,14.7,No,0


In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import StratifiedKFold

models_dct = {
    "CatBoost": CatBoostClassifier(silent=True),
    "XGBoost": XGBClassifier(enable_categorical=True),
    "MostFrequent": DummyClassifier(strategy="most_frequent"),
    "KNN": KNeighborsClassifier(),
    "NaiveBayes": GaussianNB(),
    "RandomForest": RandomForestClassifier(n_estimators=500),
    "LogRegression": LogisticRegression(max_iter=10000),
    "DecisionTree": DecisionTreeClassifier(),
}

metrics_list = [("score", roc_auc_score), ("binary", accuracy_score), ("binary", f1_score)]

In [4]:
import warnings

warnings.filterwarnings("ignore", message="is_categorical_dtype is deprecated")
warnings.filterwarnings("ignore", message="is_sparse is deprecated")

# To avoid different folds separately on each dataset
num_folds = 5
fold_generator = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=101)

summary_list = []

for model_name, model in models_dct.items():
    loguru.logger.info(f"Model: {model_name}")
    pipeline = ModelPipeline(base_model=model, features=features, metrics=metrics_list)

    folds = fold_generator.split(data, y=data["target"])
    for i, (train_fold_idx, test_fold_idx) in enumerate(folds):
        loguru.logger.info(f"Fold: {i + 1} / {num_folds}")
        train_fold = data.iloc[train_fold_idx]
        test_fold = data.iloc[test_fold_idx]
        
        pipeline.fit(train_fold)
        predictions = pipeline.predict(test_fold)

        metrics = pipeline.calculate_metrics(test_fold, predictions)

        result_dict = {
            "fold": i,
            "model": model_name,
        }
        result_dict.update(metrics)
        summary_list.append(result_dict)

summary = pd.DataFrame.from_records(summary_list)
summary.columns = ["fold", "model", "ROC AUC", "Accuracy", "F1-Score"]

summary

2023-10-07 02:31:58.342 | INFO     | __main__:<module>:13 - Model: CatBoost
2023-10-07 02:31:58.347 | INFO     | __main__:<module>:18 - Fold: 1 / 5
2023-10-07 02:32:02.963 | INFO     | __main__:<module>:18 - Fold: 2 / 5
2023-10-07 02:32:07.526 | INFO     | __main__:<module>:18 - Fold: 3 / 5
2023-10-07 02:32:11.986 | INFO     | __main__:<module>:18 - Fold: 4 / 5
2023-10-07 02:32:16.556 | INFO     | __main__:<module>:18 - Fold: 5 / 5
2023-10-07 02:32:21.109 | INFO     | __main__:<module>:13 - Model: XGBoost
2023-10-07 02:32:21.111 | INFO     | __main__:<module>:18 - Fold: 1 / 5
2023-10-07 02:32:21.287 | INFO     | __main__:<module>:18 - Fold: 2 / 5
2023-10-07 02:32:21.430 | INFO     | __main__:<module>:18 - Fold: 3 / 5
2023-10-07 02:32:21.580 | INFO     | __main__:<module>:18 - Fold: 4 / 5
2023-10-07 02:32:21.719 | INFO     | __main__:<module>:18 - Fold: 5 / 5
2023-10-07 02:32:21.859 | INFO     | __main__:<module>:13 - Model: MostFrequent
2023-10-07 02:32:21.862 | INFO     | __main__:<mo

,fold,model,ROC AUC,Accuracy,F1-Score
0,0,CatBoost,0.865602,0.8380,0.585678
1,1,CatBoost,0.868633,0.8450,0.601542
2,2,CatBoost,0.860250,0.8430,0.584656
3,3,CatBoost,0.865814,0.8380,0.564516
4,4,CatBoost,0.872338,0.8490,0.615776
5,0,XGBoost,0.852360,0.8290,0.577778
6,1,XGBoost,0.850577,0.8285,0.574969
7,2,XGBoost,0.843023,0.8275,0.567127
8,3,XGBoost,0.835028,0.8285,0.582217
9,4,XGBoost,0.846228,0.8425,0.610630
